In [1]:
import pandas as pd
import tsfresh
import os
import json
import scapy
import numpy as np
import warnings
from timeit import default_timer as timer

from scapy.all import *

warnings.filterwarnings("ignore") #ignore warnings caused by 

In [2]:
paths = [
    '../Data/network-traffic-dataset/output_bottom.csv',
    '../Data/network-traffic-dataset/output_left.csv',
    '../Data/network-traffic-dataset/output_right.csv'
]
df_w = pd.concat([pd.read_csv(p) for p in paths], ignore_index=True)

df1 = pd.read_csv('../Data/malicious/WebOS_binary.csv') #
df2 = pd.read_csv('../Data/malicious/Server_Binary.csv') #
df3 = pd.read_csv('../Data/malicious/Raspberry_Webmine_Robust.csv')
df4 = pd.read_csv('../Data/malicious/Raspberry_Binary.csv') #
df5 = pd.read_csv('../Data/malicious/Raspberry_Webmine_Aggressive.csv')
df6 = pd.read_csv('../Data/malicious/Raspberry_WebminePool_Aggressive.csv')
df7 = pd.read_csv('../Data/malicious/Server_WebminePool_Aggressive.csv') #
df32 = pd.read_csv('../Data/malicious/Server_WebminePool_Robust.csv') #
df33 = pd.read_csv('../Data/malicious/Raspberry_WebminePool_Stealthy.csv') #
df34 = pd.read_csv('../Data/malicious/Raspberry_WebminePool_Robust.csv') #
df35 = pd.read_csv('../Data/malicious/Desktop_WebminePool_Aggressive.csv') #

df_results = pd.DataFrame()

In [3]:
#################################################################
#                                                               #
#                     Filtering                                 #
#                                                               #
#################################################################

# For WebOS = 18:56:80:17:d0:ef
index_names = df1[((df1['HW_dst'] != '18:56:80:17:d0:ef') & (df1['Hw_src'] != '18:56:80:17:d0:ef'))].index
df1.drop(index_names, inplace = True)

# Big_Server_Monero_mining_data = a4:bb:6d:ac:e1:fd

index_names = df2[((df2['HW_dst'] != 'a4:bb:6d:ac:e1:fd') & (df2['Hw_src'] != 'a4:bb:6d:ac:e1:fd'))].index
df2.drop(index_names, inplace = True)

# ege_data_rasberry = dc:a6:32:67:66:4b	

index_names = df3[((df3['HW_dst'] != 'dc:a6:32:67:66:4b') & (df3['Hw_src'] != 'dc:a6:32:67:66:4b'))].index
df3.drop(index_names, inplace = True)

# Rasberry_binary_monero_mining = dc:a6:32:68:35:8a

index_names = df4[((df4['HW_dst'] != 'dc:a6:32:68:35:8a') & (df4['Hw_src'] != 'dc:a6:32:68:35:8a'))].index
df4.drop(index_names, inplace = True)

# Rasberry_network_data_2 = dc:a6:32:67:66:4b

index_names = df5[((df5['HW_dst'] != 'dc:a6:32:67:66:4b') & (df5['Hw_src'] != 'dc:a6:32:67:66:4b'))].index
df5.drop(index_names, inplace = True)

# Rasberry-Webmine = dc:a6:32:67:66:4b
index_names = df6[((df6['HW_dst'] != 'dc:a6:32:67:66:4b') & (df6['Hw_src'] != 'dc:a6:32:67:66:4b'))].index
df6.drop(index_names, inplace = True)

# Server_Webmine_Network_data = a4:bb:6d:ac:e1:fd

index_names = df7[((df7['HW_dst'] != 'a4:bb:6d:ac:e1:fd') & (df7['Hw_src'] != 'a4:bb:6d:ac:e1:fd'))].index
df7.drop(index_names, inplace = True)

# Server_%50_Mining = a4:bb:6d:ac:e1:fd

index_names = df32[((df32['HW_dst'] != 'a4:bb:6d:ac:e1:fd') & (df32['Hw_src'] != 'a4:bb:6d:ac:e1:fd'))].index
df32.drop(index_names, inplace = True)

# Rasberry_webmine_%10 = dc:a6:32:67:66:4b

index_names = df33[((df33['HW_dst'] != 'dc:a6:32:67:66:4b') & (df33['Hw_src'] != 'dc:a6:32:67:66:4b'))].index
df33.drop(index_names, inplace = True)

# Rasberry_webmine_%50 = dc:a6:32:68:35:8a

index_names = df34[((df34['HW_dst'] != 'dc:a6:32:68:35:8a') & (df34['Hw_src'] != 'dc:a6:32:68:35:8a'))].index
df34.drop(index_names, inplace = True)

# Desktop_Webmine_%100 = dc:a6:32:68:35:8a

index_names = df35[((df35['HW_dst'] != 'd8:3b:bf:8f:ba:ba') & (df35['Hw_src'] != 'd8:3b:bf:8f:ba:ba'))].index
df35.drop(index_names, inplace = True)

In [4]:
#################################################################
#                                                               #
#      Labeling Features for further calculations               #
#                                                               #
#################################################################

df1.insert(7, "Is_malicious", 1)
df2.insert(7, "Is_malicious", 1)
df3.insert(7, "Is_malicious", 1)
df4.insert(7, "Is_malicious", 1)
df5.insert(7, "Is_malicious", 1)
df6.insert(7, "Is_malicious", 1)
df7.insert(7, "Is_malicious", 1)
df32.insert(7, "Is_malicious", 1)
df33.insert(7, "Is_malicious", 1)
df34.insert(7, "Is_malicious", 1)
df35.insert(7, "Is_malicious", 1)

In [5]:
def flows_to_pseudopackets(df_flows, is_malicious: int):
    df = df_flows.copy()

    # ensure needed columns exist, then coerce to numeric
    need = ['start','end','sPackets','rPackets','sBytesAvg','rBytesAvg','sBytesSum','rBytesSum']
    for c in need:
        if c not in df.columns:
            df[c] = np.nan
    df[need] = df[need].apply(pd.to_numeric, errors='coerce')

    rows = []
    # using enumerate to get a stable integer flow_id
    for flow_id, (_, row) in enumerate(df.fillna(0).iterrows()):
        start = float(row['start']); end = float(row['end'])
        dur = max(end - start, 1e-6)

        sN = int(row['sPackets']); rN = int(row['rPackets'])

        # robust: prefer Avg if it's finite and > 0, else fallback to Sum/Packets
        def _pkt_len(avg, total, n):
            if pd.notna(avg) and avg > 0:
                return float(avg)
            return (float(total) / n) if (pd.notna(total) and n > 0) else 0.0

        s_len = _pkt_len(row['sBytesAvg'], row['sBytesSum'], sN)
        r_len = _pkt_len(row['rBytesAvg'], row['rBytesSum'], rN)

        N = sN + rN
        if N == 0:
            continue

        # distribute timestamps across the flow window
        for k in range(sN):
            t = start + dur * (k + 0.5) / N
            rows.append({'flow_id': flow_id, 'Time': t, 'Length': s_len, 'Is_malicious': is_malicious})
        for k in range(rN):
            t = start + dur * (sN + k + 0.5) / N
            rows.append({'flow_id': flow_id, 'Time': t, 'Length': r_len, 'Is_malicious': is_malicious})

    out = pd.DataFrame(rows)
    if out.empty:
        return out

    out = out.sort_values(['flow_id','Time']).reset_index(drop=True)

    # window index (groups of 10 packets within each flow)
    out['win'] = out.groupby('flow_id').cumcount() // 10

    # unique integer series id per (flow_id, win) WITHOUT bit operations
    out['id'] = pd.factorize(list(zip(out['flow_id'].astype('int64'), out['win'].astype('int64'))))[0].astype('int64')

    return out

In [6]:
def run_process(a,b,x):
    from tsfresh import extract_features, select_features
    from tsfresh.utilities.dataframe_functions import impute
    from tsfresh import extract_features
    from tsfresh.feature_selection.relevance import calculate_relevance_table

    df_malicious = a.copy()
    df_benign    = b.copy()

    for df in (df_malicious, df_benign):
        if 'id' not in df.columns:
            df.reset_index(drop=True, inplace=True)
            df['id'] = (df.index // 10).astype(int)

    tf1=tsfresh.extract_features(df_malicious,impute_function=impute, column_kind='Is_malicious',
                                 column_id='id',column_sort="Time",column_value = "Length")
    tf1['class']= 1

    
    
    tf2=tsfresh.extract_features(df_benign,impute_function=impute, column_kind='Is_malicious',
                                 column_id='id',column_sort="Time",column_value = "Length")
    tf2['class']= 0


    tf2.columns = tf1.columns

    features=pd.concat([tf1,tf2])


    features2 = features.copy()
    features2.reset_index(drop=True, inplace=True)
    
    y = pd.Series(data = features2['class'], index=features2.index)
    
    from tsfresh.examples import load_robot_execution_failures
    from tsfresh import extract_features, select_features
    from tsfresh.feature_selection.relevance import calculate_relevance_table

    relevance_table = calculate_relevance_table(features2, y)
    relevance_table = relevance_table[relevance_table.relevant]
    relevance_table.sort_values("p_value", inplace=True)

    relevance_table
    
    best_features = relevance_table[relevance_table['p_value'] <= 0.05]

    df_ML = pd.DataFrame()

    for pkt in best_features:
        df_ML[best_features.feature] = features[best_features.feature]

    final = ML_Process(df_ML,x)

    return final

In [7]:
def ML_Process(df_ML,x):
    df_results = x.copy() 
    print('let the ml starts')
  
    from sklearn import neighbors, metrics
    from sklearn.preprocessing import LabelEncoder

    #X = df_finalized[['Time', 'Length','Protocol']].values
    X = df_ML.drop('class',axis=1).to_numpy()
    #y = df_finalized[['Is_malicious']]
    y = df_ML['class'].to_numpy()


    #print(X,y)
    
    from sklearn.model_selection import train_test_split
    Le = LabelEncoder()
    for i in range(len(X[0])):
        X[:, i] = Le.fit_transform(X[:, i])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=8675309)

    
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.naive_bayes import GaussianNB
    #from xgboost import XGBClassifier
    from sklearn import model_selection
    from sklearn.utils import class_weight
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    import numpy as np
    import pandas as pd
    y_train = y_train.ravel()
    dfs = []
    models = [
          ('LogReg', LogisticRegression()), 
          #('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
          ('SVM', SVC()), 
          ('GNB', GaussianNB())
          #('XGB', XGBClassifier())
            ]
    results = []
    names = []
    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
    target_names = ['malignant', 'benign']
    for name, model in models:
        kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, 
                                                    scoring=scoring)

        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names))
        results.append(cv_results)
        names.append(name)
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        dfs.append(this_df)
        df_resulta = pd.concat([df_results] + dfs, ignore_index=True)
        final = pd.concat(dfs, ignore_index=True)
        print(final)

    return(final)


In [9]:
def get_n_groups(df: pd.DataFrame, group_col: str, n_groups: int, random_state: int = 42) -> pd.DataFrame:
    """
    Randomly pick n_groups distinct values of `group_col` and return all rows that belong to them.
    Useful when your units are flows or 10-packet windows (e.g., group_col='id' or 'flow_id').
    """
    uniq = df[group_col].drop_duplicates()
    n_pick = min(int(n_groups), len(uniq))
    chosen = uniq.sample(n=n_pick, random_state=random_state)
    return df[df[group_col].isin(chosen)].reset_index(drop=True)

In [11]:
# filter the benign data
df_benign_flows = df_w[df_w['NST_B_Label'] == 0].copy()
print("Benign flows:", len(df_benign_flows))

Benign flows: 36648


In [12]:
df_benign_packets = flows_to_pseudopackets(df_benign_flows, is_malicious=0)
print(df_benign_packets.head())

   flow_id          Time  Length  Is_malicious  win  id
0        0  1.678441e+09    59.0             0    0   0
1        0  1.678441e+09    59.0             0    0   0
2        0  1.678441e+09    59.0             0    0   0
3        0  1.678441e+09    59.0             0    0   0
4        0  1.678441e+09    59.0             0    0   0


In [14]:
df_benign = get_n_groups(df_benign_packets, group_col='id', n_groups=1200)
len(df_benign)

10260

In [ ]:
# Scenario 2; Throttles
    
    # 1) THR: %10 (Stealthy)

df_malicious = pd.concat([df33])
df_benign = get_n_groups(df_benign_packets, group_col='id', n_groups=1150)

# if df_malicious lacks a per-10-packet window id, add one:
if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  # OK if malicious packets are contiguous

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_THR_10_s2 = run_process(df_malicious,df_benign,df_results)


end = timer()
print(end - start)

malicious: 9880
benign: 8345
0 NAN in malicious!
0 NAN in benign!
After droppping NAN rows: 
malicious: 9880
benign: 8345


Feature Extraction: 100%|██████████| 30/30 [00:17<00:00,  1.70it/s]


let the ml starts
LogReg
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00       242
      benign       1.00      1.00      1.00       250

    accuracy                           1.00       492
   macro avg       1.00      1.00      1.00       492
weighted avg       1.00      1.00      1.00       492

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  0.026000    0.011999        1.00000                 1.000000   
1  0.011002    0.008001        1.00000                 1.000000   
2  0.009997    0.008001        0.99661                 0.996631   
3  0.010998    0.009003        1.00000                 1.000000   
4  0.014999    0.011003        1.00000                 1.000000   

   test_recall_weighted  test_f1_weighted  test_roc_auc   model  
0               1.00000          1.000000      1.000000  LogReg  
1               1.00000          1.000000      1.000000  LogReg  
2               0.99661          0.996609      

Exception in thread Thread-16 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\T-Gamer\miniconda3\envs\ids-project\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\T-Gamer\miniconda3\envs\ids-project\lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\T-Gamer\miniconda3\envs\ids-project\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\T-Gamer\miniconda3\envs\ids-project\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\T-Gamer\miniconda3\envs\ids-project\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc6 in position 72: invalid continuation byte


KNN
              precision    recall  f1-score   support

   malignant       0.99      1.00      1.00       242
      benign       1.00      0.99      1.00       250

    accuracy                           1.00       492
   macro avg       1.00      1.00      1.00       492
weighted avg       1.00      1.00      1.00       492

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  0.026000    0.011999       1.000000                 1.000000   
1  0.011002    0.008001       1.000000                 1.000000   
2  0.009997    0.008001       0.996610                 0.996631   
3  0.010998    0.009003       1.000000                 1.000000   
4  0.014999    0.011003       1.000000                 1.000000   
5  0.003999    3.313967       0.996622                 0.996644   
6  0.002002    0.027999       1.000000                 1.000000   
7  0.001999    0.028998       0.996610                 0.996631   
8  0.000999    0.027000       0.989831                 0.990037   

In [ ]:
# 2) THR: %50 (Robust)

df_malicious = pd.concat([df3,df34])
df_benign = get_n_groups(df_benign_packets, group_col='id', n_groups=1200)

if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_THR_50_s2 = run_process(df_malicious,df_benign,df_results)

end = timer()
print(end - start)

In [ ]:
 # 3) THR: %100 (Aggressive)
    
df_malicious = pd.concat([df1,df2,df4,df5,df6,df7,df35])
df_benign    = df_benign_packets

if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_THR_100_s2 = run_process(df_malicious,df_benign,df_results)


end = timer()
print(end - start)


In [ ]:
## S3; In-browser VS Binary

#1) In-Browser
df_malicious = pd.concat([df3,df5,df6,df7,df32,df33,df34,df35])
df_benign = get_n_groups(df_benign_packets, group_col='id', n_groups=35200)

if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_In_s3 = run_process(df_malicious,df_benign,df_results)


end = timer()
print(end - start)

In [ ]:
#2) Binary
df_malicious = pd.concat([df1,df2,df4])
df_benign    = df_benign_packets

if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_Host_s3 = run_process(df_malicious,df_benign,df_results)


end = timer()
print(end - start)